# MODIS



MODIS Terra: https://lpdaac.usgs.gov/products/mod11c1v006/

MODIS Acqua: https://lpdaac.usgs.gov/products/myd11c1v061/

Docs: https://lpdaac.usgs.gov/documents/118/MOD11_User_Guide_V6.pdf

In [1]:
! wget https://e4ftl01.cr.usgs.gov/MOLA/MYD11C1.061/2018.04.19/MYD11C1.A2018109.061.2021330052306.hdf -O /network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/MODIS.hdf

--2022-03-18 11:14:00--  https://e4ftl01.cr.usgs.gov/MOLA/MYD11C1.061/2018.04.19/MYD11C1.A2018109.061.2021330052306.hdf
Resolving e4ftl01.cr.usgs.gov (e4ftl01.cr.usgs.gov)... 152.61.133.130, 2001:49c8:4000:127d::133:130
Connecting to e4ftl01.cr.usgs.gov (e4ftl01.cr.usgs.gov)|152.61.133.130|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://urs.earthdata.nasa.gov/oauth/authorize?scope=uid&app_type=401&client_id=ijpRZvb9qeKCK5ctsn75Tg&response_type=code&redirect_uri=https%3A%2F%2Fe4ftl01.cr.usgs.gov%2Foauth&state=aHR0cHM6Ly9lNGZ0bDAxLmNyLnVzZ3MuZ292L01PTEEvTVlEMTFDMS4wNjEvMjAxOC4wNC4xOS9NWUQxMUMxLkEyMDE4MTA5LjA2MS4yMDIxMzMwMDUyMzA2LmhkZg [following]
--2022-03-18 11:14:08--  https://urs.earthdata.nasa.gov/oauth/authorize?scope=uid&app_type=401&client_id=ijpRZvb9qeKCK5ctsn75Tg&response_type=code&redirect_uri=https%3A%2F%2Fe4ftl01.cr.usgs.gov%2Foauth&state=aHR0cHM6Ly9lNGZ0bDAxLmNyLnVzZ3MuZ292L01PTEEvTVlEMTFDMS4wNjEvMjAxOC4wNC4xOS9NWUQxMUMxLkEyMDE4MTA5LjA2

In [2]:
import xarray as xr
import rioxarray as rxr

modis_xarray= rxr.open_rasterio('/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/MODIS.hdf',masked=True)

In [54]:
modis_xarray['Day_view_time']

<xarray.DataArray 'Day_view_time' (band: 1, y: 3600, x: 7200)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 89.97 89.92 89.88 89.82 ... -89.88 -89.93 -89.98
  * x            (x) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    scale_factor:  0.2
    add_offset:    0.0
    long_name:     Time of daytime LST observation (UTC)
    units:         hrs

In [35]:
modis_df = modis_xarray.to_dataframe().reset_index()

In [41]:
lite = modis_df[['y','x', 'LST_Day_CMG','Day_view_time', 'LST_Night_CMG', 'Night_view_time']]

In [48]:
L = lite.dropna().copy()

In [59]:
L['T_Day'] = L['LST_Day_CMG']*0.02
L['T_Night'] = L['LST_Night_CMG']*0.02
L['Time_Day'] = L['Day_view_time']*0.2
L['Time_Night'] = L['Night_view_time']*0.2
L['Time_Day_UTC'] = L['Time_Day'] - L['x']/15

In [60]:
L

,y,x,LST_Day_CMG,Day_view_time,LST_Night_CMG,Night_view_time,T_Day,T_Night,Time_Day,Time_Night,Time_Day_UTC
910125,83.675,-33.725,12387.0,53.0,12266.0,36.0,247.739990,245.319992,10.6,7.2,12.848334
910126,83.675,-33.675,12387.0,53.0,12266.0,36.0,247.739990,245.319992,10.6,7.2,12.845000
910127,83.675,-33.625,12387.0,53.0,12266.0,36.0,247.739990,245.319992,10.6,7.2,12.841667
910128,83.675,-33.575,12387.0,53.0,12266.0,36.0,247.739990,245.319992,10.6,7.2,12.838334
910129,83.675,-33.525,12387.0,53.0,12266.0,36.0,247.739990,245.319992,10.6,7.2,12.835000
...,...,...,...,...,...,...,...,...,...,...,...
25919833,-89.975,171.675,10529.0,29.0,10856.0,51.0,210.580002,217.119995,5.8,10.2,-5.645000
25919834,-89.975,171.725,10529.0,29.0,10856.0,51.0,210.580002,217.119995,5.8,10.2,-5.648333
25919835,-89.975,171.775,10529.0,29.0,10856.0,51.0,210.580002,217.119995,5.8,10.2,-5.651666
25919836,-89.975,171.825,10529.0,29.0,10856.0,51.0,210.580002,217.119995,5.8,10.2,-5.655000


In [44]:
lite.query('y == -69.983333')

,y,x,LST_Day_CMG,Day_view_time,LST_Night_CMG,Night_view_time


In [32]:
modis_xarray.LST_Day_CMG.data.dropna()

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'

In [11]:
t = modis_xarray.Day_view_time.data

In [15]:
tt = t.flatten()

In [16]:
sum(tt)

nan

In [25]:
import numpy as np
tnn = t[~np.isnan(t)]*0.2

In [26]:
len(tnn)

6429550

In [27]:
min(tnn)

0.0

In [28]:
max(tnn)

24.0

In [29]:
np.unique(tnn)

array([ 0.       ,  0.2      ,  0.4      ,  0.6      ,  0.8      ,
        1.       ,  1.2      ,  1.4      ,  1.6      ,  1.8000001,
        2.       ,  2.2      ,  2.4      ,  2.6000001,  2.8      ,
        3.       ,  3.2      ,  3.4      ,  3.6000001,  3.8      ,
        4.       ,  4.2000003,  4.4      ,  4.6      ,  4.8      ,
        5.       ,  5.2000003,  5.4      ,  5.6      ,  5.8      ,
        6.       ,  6.2000003,  6.4      ,  6.6      ,  6.8      ,
        7.       ,  7.2000003,  7.4      ,  7.6      ,  7.8      ,
        8.       ,  8.2      ,  8.400001 ,  8.6      ,  8.8      ,
        9.       ,  9.2      ,  9.400001 ,  9.6      ,  9.8      ,
       10.       , 10.2      , 10.400001 , 10.6      , 10.8      ,
       11.       , 11.2      , 11.400001 , 11.6      , 11.8      ,
       12.       , 12.2      , 12.400001 , 12.6      , 12.8      ,
       13.       , 13.2      , 13.400001 , 13.6      , 13.8      ,
       14.       , 14.2      , 14.400001 , 14.6      , 14.8   